# Install Dependencies

In [31]:
#!pip install ultralytics
#!pip install roboflow

# Import libraries

In [1]:
from ultralytics import YOLO
from roboflow import Roboflow
import shutil
import torch
import os

# Get Dataset


In [33]:
# Import custom dataset from roboflow
rf = Roboflow(api_key="1BUdGcTVp8UVzqdnYevS")
project = rf.workspace("computervision-jifim").project("hockey-playerdetection")
version = project.version(3)
dataset = version.download("yolov8")
dataset.location = dataset.location + "/data.yaml"

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.70, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to hockey-playerdetection-3 in yolov8:: 100%|██████████| 2170/2170 [00:03<00:00, 627.94it/s]


# Organize Files

In [34]:
# Organize folders in the format yolo expects
shutil.move('hockey-playerdetection-5/train', 'hockey-playerdetection-5/hockey-playerdetection-5/train')
shutil.move('hockey-playerdetection-5/valid', 'hockey-playerdetection-5/hockey-playerdetection-5/valid')

'hockey-playerdetection-3/hockey-playerdetection-3/valid'

# Clear Cache

In [ ]:
torch.cuda.empty_cache()

# Optimize Hyperparameters

In [35]:
# Check if there is a previous model we can train off
build_on_previous_model = os.path.isfile("../models/last.pt")

# If there is a previous model, we will leverage transfer learning (start optimizing from pretrained *.pt model)
model = YOLO("../models/last.pt" if build_on_previous_model else "yolov8l.pt")

calc_optimal_hyper_param = False    # do we want to calculate the optimal hyperparameters

# If true, tune hyperparameters on dataset
if (calc_optimal_hyper_param):
    model.tune(data=dataset.location, epochs=30, iterations=300, optimizer="AdamW", plots=False, save=False, val=False)

# Train

In [30]:
# Check if we have optimal hyperparameters
has_optimal_hyper_param = os.path.isfile("../models/best_hyperparameters.yaml")

if (has_optimal_hyper_param):
    results = model.train(data=dataset.location, epochs=100, imgsz=640, cfg="../models/best_hyperparameters.yaml")
else:
    results = model.train(data=dataset.location, epochs=100, imgsz=640)

False
